In [11]:
# kill conections opened
from multiprocessing import Process
import carla

def do_stuff(x):
    host = 'localhost'
    port = 2000
    carla_client = carla.Client(host=host, port=port)
    carla_client.set_timeout(5)
    carla_world = carla_client.get_world()
    # etc

p = Process(target=do_stuff)
p.start()
p.join()
if p.is_alive():
    p.kill()

In [3]:
# all imports
import carla
import random
import cv2
import numpy as np

In [8]:
client = carla.Client('localhost', 2000)

client.load_world('Town05')

In [5]:
# define envirimental and get possible places to spawn a car
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()

In [6]:
# Look for a blueprint of a mini car
vehicle_bp = world.get_blueprint_library().filter('*mini*')

In [7]:
# spawn a car in a random location

start_point = random.choice(spawn_points)
vehicle = world.try_spawn_actor(vehicle_bp[0],start_point)

In [8]:
# Move the view to the car

spectator = world.get_spectator()
start_point.location.z = start_point.location.z+1
spectator.set_transform(start_point)

In [9]:
vehicle.set_autopilot(True)

In [10]:
# setting RGB 

CAMERA_POS_Z = 1.6
CAMERA_POS_X = 0.9

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x','640')
camera_bp.set_attribute('image_size_y','360')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z, x = CAMERA_POS_X))
camera = world.spawn_actor(camera_bp,camera_init_trans, attach_to= vehicle)

def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image':np.zeros((image_h,image_w,4))}
camera.listen(lambda image: camera_callback(image,camera_data))

In [11]:
# grab a snapshot from camerar an show in a pop-up windon
# while():
#     img = camera_data['image']ee
#     cv2.imshow('RGB Camera ', img)
#     cv2.waitKey(2)

In [12]:
# utility script of destruction

camera.stop()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()